In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from deepface import DeepFace
from deepface.commons import functions

import lpips_tf

# Algorithm 1: Ensemble attack training procedure
Inputs:
- Victim Models (F)
- Image Dataset (X)

Outputs:
- Perturbation Engine (g_theta)

HyperParams:
- Learning rate (alpha)
- L_inf bound (epsilon)
- Lpips loss coefficient (lambda)

## Inputs

### Victim Models (F)

In [ ]:
victim_models_labels = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "SFace"
]

F = set()

for model_name in victim_models_labels:
  F.add((DeepFace.build_model(model_name=model_name).model, functions.find_target_size(model_name)))
  
print(len(F))

### Image Dataset (X)

In [ ]:
ds, info = tfds.load('lfw',
                      with_info=True,
                      download=True,
                      as_supervised=False)

In [ ]:
ds = ds['train']

## HyperParams

In [ ]:
alpha = tf.Variable(0.2, trainable=False, name='alpha')

In [ ]:
eps = tf.Variable(0.5, trainable=False, name='epsilon')

In [ ]:
lda = tf.Variable(0.2, trainable=False, name='lambda')

### Initialized ATN ($N_{theta}$)

In [ ]:
model = UNet(input_size=(128,128,3), n_filters=32, n_classes=3)

In [ ]:
model.summary()

## Algorithm

### Adversarial Noise
$$ x_{adv} \leftarrow clip_{[0,1]}(x + \epsilon \cdot \tanh(N_\theta(x))) $$

In [ ]:
def addAdversarialNoise(x, eps, atn, training=True):
  xadv = atn(x, training=training)
  xadv = tf.tanh(xadv)
  xadv = tf.multiply(eps, xadv)
  xadv = tf.add(x, xadv)
  xadv = tf.clip_by_value(xadv, 0, 1)
  return xadv
  # return tf.clip_by_value(tf.add(x, tf.multiply(eps, tf.tanh(atn(x)))), 0, 1)

### Embeddings Loss
$$
loss \leftarrow \dfrac{1}{\left\|\mathbb{F}\right\|} \sum^{\mathbb{F}}_{f} - \dfrac{f(x) \cdot f(x_{adv})} {\left\| f(x)\right\|_{2}\left\| f(x_{adv})\right\|_{2}}
$$

In [ ]:
def fCosineDistance(x, x_adv, f):
  emb_t = f(x)
  emb_adv = f(x_adv)
  dist = tf.keras.losses.cosine_similarity(emb_t, emb_adv, axis=1)
  dist = tf.negative(dist)
  return dist

In [ ]:
def FLoss(x, x_adv, loss, F):
  N = len(F)
  for f in F:
    model = f[0]
    in_shape = f[1]
    #TODO: convert to right shape
    loss = tf.add(loss, fCosineDistance(model, x, x_adv))
  loss = tf.divide(loss, N)
  return loss

### Perceptual Loss ($L_{pips}$)
$$
loss \leftarrow loss + \lambda L_{pips}(x_{adv}, x)
$$

In [ ]:
def LpipsLoss(x, x_adv, loss, lda):
  dist = lpips_tf.lpips(x_adv, x, model='net-lin', net='alex')
  dist = tf.multiply(lda, dist)
  loss = tf.add(loss, dist)
  return loss

### Loss Function

In [ ]:
def atnLoss(x, x_adv, F, lda):
  loss = tf.Variable(0, trainable=False, name='loss')
  loss = FLoss(x, x_adv, loss, F)
  loss = LpipsLoss(x, x_adv, loss, lda)
  return loss

## Train Loop

In [ ]:
def train(X, F, model, optimizer, eps, lda):
  for epoch in range(epoch):
    print("\nEpoch: %d" % (epoch,))
    for step, (x_batch) in enumerate(X):
      # Open GradientTape to record ops run during forward pass for auto-differentiation
      with tf.GradientTape() as tape:
        # Forward pass of layer.
        # Ops applied recorded on GradientTape
        x_adv = addAdversarialNoise(x_batch, eps, model, True)
        loss = atnLoss(x_batch, x_adv, F, lda)
      
      # Use gradient tape to retrieve grads of trainable variables wrt loss
      grads = tape.gradient(loss, model.trainable_weights)
      
      # Gradient descent, update variables to minimize loss.
      optimizer.apply_gradients(zip(grads, model.trainable_weights))
      
      if step % 200 == 0:
        print(
          "Training loss (for one batch) at step %d: %.4f"
          % (step, float(loss))
        )

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=atnLoss)

In [ ]:
from unet import AuraMask

In [ ]:
model = AuraMask(32, 3)

In [ ]:
model.build((None, 512,512,3))

In [ ]:
model.summary()

In [ ]:
from enum import Enum

class FaceEmbeddingsEnum(Enum):
  VGGFACE = "VGG-Face"
  FACENET = "Facenet"
  FACENET512 = "Facenet512"
  OPENFACE = "OpenFace"
  DEEPFACE = "DeepFace"
  DEEPID = "DeepID"
  ARCFACE = "ArcFace"
  SFACE = "SFace"
  def get_model(self):
    return DeepFace.build_model(model_name=self.value).model
  def get_target_size(self):
    return functions.find_target_size(model_name=self.value)

In [ ]:
def build_F(targets: list[FaceEmbeddingsEnum]) -> set[(tf.Model, tuple)]:
  F = set()
  for model_label in targets:
    F.add(
      (
        model_label.get_model(),
        model_label.get_target_size(),
      )
    )
  return F